This notebook is adapted for running on a local machine

# 1. Data prep

In [27]:
from config import *

In [28]:
#--- Loading the filtered metadata ---

out_path = '/Users/kamilkon/Desktop/Neuro120FP/meta_val_filtered.pkl'
with open(out_path, 'rb') as f:
    meta_kept = pickle.load(f)
print(f"Loaded {len(meta_kept)} samples from {out_path}.")

Loaded 629 samples from /Users/kamilkon/Desktop/Neuro120FP/meta_val_filtered.pkl.


In [29]:
#--- Functions for computing RDM and the Mantel test ---
def compute_rdm(feats, metric='correlation'):
    """
    Inputs:
        feats = a 2D array of shape (n_samples, n_features)
        metric = a string indicating the distance metric to use
    Outputs:
        rdm = a 2D array of shape (n_samples, n_samples) containing the pairwise distances
    """

    return squareform(pdist(feats, metric=metric))


def mantel_test(rdm1, rdm2, n_perm=500):
    """
    Inputs:
        rdm1 = a 2D array of shape (n_samples, n_samples) containing the human RDM
        rdm2 = a 2D array of shape (n_samples, n_samples) containing the CNN RDM
        n_perm = the number of permutations to perform
    Outputs:
        r0 = the observed correlation coefficient
        pval = the p-value of the test
    """

    iu = np.triu_indices_from(rdm1, k=1)
    v1 = rdm1[iu]; v2 = rdm2[iu]
    r0, _ = pearsonr(v1, v2)
    perm_rs = []
    idx = np.arange(rdm1.shape[0])
    for _ in range(n_perm):
        np.random.shuffle(idx)
        perm = rdm2[np.ix_(idx, idx)]
        perm_rs.append(pearsonr(v1, perm[iu])[0])
    perm_rs = np.array(perm_rs)
    pval = (np.sum(perm_rs >= r0) + 1)/(n_perm+1)
    return r0, pval

# 2. Computing features from the fMRI and the CNN

In [ ]:
#--- Computing features ---

# Instantiating the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model  = Bigger3DCNN(in_channels=1, num_classes=5).to(device)
model.load_state_dict(torch.load('bestmodel.pth',
                                 map_location=device))
model.eval()

# Building up the features of the RDMs
features_human = []
features_cnn   = []

for entry in meta_kept:
    subj = entry['subject']
    run  = entry['run_idx']
    fp = entry.get('filepath') or (f"Data/ds000113/sub-{subj}/ses-movie/func/sub-{subj}_ses-movie_task-movie_run-{run}_bold.nii.gz")
    img   = nib.load(fp)
    vol   = img.get_fdata()[..., entry['TR_local']].astype(np.float32)
    vol_n = (vol - vol.mean())/(vol.std()+1e-6)

    # Human features => flattened volumes
    features_human.append(vol_n.ravel())

    # CNN features => logits
    x = torch.from_numpy(vol_n[None, None]).to(device)
    with torch.no_grad():
        logits = model(x)               
    features_cnn.append(logits.cpu().numpy().ravel())

features_human = np.stack(features_human, 0)  # (N, X*Y*Z)
features_cnn   = np.stack(features_cnn,   0)  # (N, num_classes)

print("Human feats:", features_human.shape)
print("CNN   feats:", features_cnn.shape)

# PCA reducing the human features
pca = PCA(n_components=50, random_state=0).fit(features_human)
human_pca = pca.transform(features_human)     # (N, 50)

# Computing the RDMs
rdm_human = compute_rdm(human_pca, metric='correlation')
rdm_cnn   = compute_rdm(features_cnn, metric='correlation')

# Mantel test
r, p = mantel_test(rdm_human, rdm_cnn, n_perm=1000)
print(f"Mantel correlation = {r:.3f}, p = {p:.3f}")

In [32]:
#--- Recalling labels from metadata ---
unique_labels = sorted({ entry['label'] for entry in meta_kept })
label_to_int  = { lab:i for i, lab in enumerate(unique_labels) }
inv_label_map = { i:lab for lab,i in label_to_int.items() }

# 3. Creating comparative RDMs

In [33]:
#--- Helper function for extracting features ---

def extract_feats(vol3d):
    """ 
    Inputs:
        vol3d = a 3D numpy array of shape (X, Y, Z)
    Outputs:
        out = a 1D numpy array of shape (256,) containing the features
    """
    
    x = torch.from_numpy(vol3d[None,None]).float().to(device)
    with torch.no_grad():
        out = F.relu6(model.bn1(model.conv1(x)))
        out = model.ds1(out); out = model.ds2(out)
        out = model.ds3(out); out = model.ds4(out)
        out = model.pool(out).view(1,-1)   # (1,256)
    return out.cpu().numpy()[0]            # (256,)

In [ ]:
#--- Extracting features to build an actual matrix ---

Xh = np.vstack([ e['volume'].ravel() for e in meta_kept ])
yh = np.array([ label_to_int[e['label']] for e in meta_kept ])
mask_var = Xh.std(0) > 0
Xh = (Xh[:,mask_var] - Xh[:,mask_var].mean(0)) / (Xh[:,mask_var].std(0)+1e-6)

# Human RDM
human_means = np.vstack([ Xh[yh==lbl].mean(0) for lbl in range(5) ])
rdm_h = squareform(pdist(human_means, metric='correlation'))

# CNN RDM
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device).eval()
## Building a feature matrix
Xc = np.vstack([ extract_feats(e['volume']) for e in meta_kept ])
yc = np.array([ label_to_int[e['label']] for e in meta_kept ])
## Normalizing the features
Xc = (Xc - Xc.mean(0)) / (Xc.std(0)+1e-6)
## Averaging over emotion
cnn_means = np.vstack([ Xc[yc==lbl].mean(0) for lbl in range(5) ])
rdm_c = squareform(pdist(cnn_means, metric='correlation'))

# Calculating the Mantel test
idx = np.triu_indices(5, k=1)
r,p = pearsonr(rdm_h[idx], rdm_c[idx])

# Plotting the RDMs
emotion_names = [inv_label_map[i] for i in range(5)]
fig, (ax1,ax2) = plt.subplots(1,2,figsize=(8,4),constrained_layout=True)
for ax, rdm, title in zip((ax1,ax2),(rdm_h,rdm_c),("Human RDM","CNN RDM")):
    im = ax.imshow(rdm, vmin=0, vmax=1, cmap="RdBu_r")
    ax.set_title(title)
    ax.set_xticks(range(5)); ax.set_xticklabels(emotion_names, rotation=45)
    ax.set_yticks(range(5)); ax.set_yticklabels(emotion_names)
    ax.set_xticks(np.arange(-.5,5), minor=True)
    ax.set_yticks(np.arange(-.5,5), minor=True)
    ax.grid(which="minor", color="k", lw=1)
for spine in ax2.spines.values(): spine.set_visible(False)
cbar = fig.colorbar(im, ax=(ax1,ax2), fraction=.03, pad=0.02)
cbar.set_label("dist = 1–Pearson r", rotation=270, labelpad=12)
fig.suptitle(f"Mantel r={r:.3f}, p={p:.3f}", y=1.05, fontsize=14)
plt.show()
